In [ ]:
from google.colab import drive
drive.mount('/content/drive')

In [ ]:
import os
import numpy as np
import cv2
import tensorflow as tf
import sklearn
import matplotlib.pylab as plt
import pandas as pd

from tensorflow.keras import layers, models
from sklearn.metrics import roc_curve, auc, accuracy_score, confusion_matrix, classification_report

In [ ]:
path = "..." # insert your own path of ADNI-2 data with normalization, automated cropping, registration w.r.t. AD/CN avg template and brain extraction applied
             # ADNI-2 data can not be provided here, but prediction and ground truth scores are provided in this repository 

X = [] 
Y = [] 

dir = path 
classes = ["AD","CN"] 
classes_list = os.listdir(dir) 
classes_list.sort() 

full_path_AD = os.path.join(dir, classes_list[0]) 
samples_AD = os.listdir(full_path_AD) 
samples_AD.sort() 
for a in samples_AD: 
  b = os.path.join(full_path_AD, a) 
  data = np.load(b)
  data = data.astype('float32') 
  temp = [] 
  for i in range(84,134):
    temp.append(cv2.resize(data[:,:,i], (192,147))) 
  X.append(temp) 
  y = [0]*len(classes_list) 
  y[1] = 1 
  Y.append(y) 

full_path_CN = os.path.join(dir, classes_list[1]) 
samples_CN = os.listdir(full_path_CN) 
samples_CN.sort()
for a in samples_CN: 
  b = os.path.join(full_path_CN, a)
  data = np.load(b)
  data = data.astype('float32')
  temp = []
  for i in range(84,134): 
    temp.append(cv2.resize(data[:,:,i], (192,147)))
  X.append(temp)
  y = [0]*len(classes_list)
  y[0] = 1 
  Y.append(y) 
  
X = np.asarray(X) 
X = X[...,np.newaxis] 
Y = np.asarray(Y) 

print(X.shape)
print(Y.shape)

In [ ]:
for j in range (0,65):
  for i in range (0,49):
    plt.imshow(X[j,i,:,:,0].transpose(),cmap='gray',origin='lower')
    plt.show()
    i+=1
    print(j+1)
  j+=1

In [ ]:
model = models.Sequential()
model.add(layers.ConvLSTM2D(filters=8, 
                            kernel_size=(3,3), 
                            return_sequences=False, 
                            data_format="channels_last", 
                            input_shape=(50, 147, 192, 1)))
model.add(layers.Dropout(0.6)) 
model.add(layers.Flatten())
model.add(layers.Dense(256, activation="relu"))
model.add(layers.Dropout(0.6)) 
model.add(layers.Dense(2, activation="softmax"))

model.summary()

opt = tf.keras.optimizers.SGD(learning_rate=0.005) 
model.compile(loss='binary_crossentropy', optimizer=opt, metrics=["accuracy"])

In [ ]:
model = models.load_model(".../model-best.h5")

### Female group

In [ ]:
pred = pd.read_excel(r".../pred-female.xlsx")
pred = np.asarray(pred)

Y = pd.read_excel(r".../ground-female.xlsx")
Y = np.asarray(Y)

y_true = Y.argmax(axis=-1)
y_pred = pred.argmax(axis=-1)
report = classification_report(y_true, y_pred, target_names=['CN','AD'], output_dict=True)
print(classification_report(y_true, y_pred, target_names=['CN','AD']))
print(confusion_matrix(y_true, y_pred))

mean_fpr = np.linspace(0,1,100)
fpr, tpr, thr = roc_curve(y_true, pred[:,1], pos_label=1)
roc_auc = auc(fpr, tpr)

              precision    recall  f1-score   support

          CN       0.95      0.78      0.86        23
          AD       0.55      0.86      0.67         7

    accuracy                           0.80        30
   macro avg       0.75      0.82      0.76        30
weighted avg       0.85      0.80      0.81        30

[[18  5]
 [ 1  6]]


### Male group

In [ ]:
pred = pd.read_excel(r".../pred-male.xlsx")
pred = np.asarray(pred)

Y = pd.read_excel(r".../ground-male.xlsx")
Y = np.asarray(Y)

y_true = Y.argmax(axis=-1)
y_pred = pred.argmax(axis=-1)
report = classification_report(y_true, y_pred, target_names=['CN','AD'], output_dict=True)
print(classification_report(y_true, y_pred, target_names=['CN','AD']))
print(confusion_matrix(y_true, y_pred))

mean_fpr = np.linspace(0,1,100)
fpr, tpr, thr = roc_curve(y_true, pred[:,1], pos_label=1)
roc_auc = auc(fpr, tpr)

              precision    recall  f1-score   support

          CN       0.80      0.80      0.80        10
          AD       0.92      0.92      0.92        24

    accuracy                           0.88        34
   macro avg       0.86      0.86      0.86        34
weighted avg       0.88      0.88      0.88        34

[[ 8  2]
 [ 2 22]]


### < 65 y. o. group

In [ ]:
pred = pd.read_excel(r".../pred-agemin65.xlsx")
pred = np.asarray(pred)

Y = pd.read_excel(r".../ground-agemin65.xlsx")
Y = np.asarray(Y)

y_true = Y.argmax(axis=-1)
y_pred = pred.argmax(axis=-1)
report = classification_report(y_true, y_pred, target_names=['CN','AD'], output_dict=True)
print(classification_report(y_true, y_pred, target_names=['CN','AD']))
print(confusion_matrix(y_true, y_pred))

mean_fpr = np.linspace(0,1,100)
fpr, tpr, thr = roc_curve(y_true, pred[:,1], pos_label=1)
roc_auc = auc(fpr, tpr)

              precision    recall  f1-score   support

          CN       0.50      1.00      0.67         1
          AD       1.00      0.80      0.89         5

    accuracy                           0.83         6
   macro avg       0.75      0.90      0.78         6
weighted avg       0.92      0.83      0.85         6

[[1 0]
 [1 4]]


### 65 - 75 y. o. group

In [ ]:
pred = pd.read_excel(r".../pred-age6575.xlsx")
pred = np.asarray(pred)

Y = pd.read_excel(r".../ground-age6575.xlsx")
Y = np.asarray(Y)

y_true = Y.argmax(axis=-1)
y_pred = pred.argmax(axis=-1)
report = classification_report(y_true, y_pred, target_names=['CN','AD'], output_dict=True)
print(classification_report(y_true, y_pred, target_names=['CN','AD']))
print(confusion_matrix(y_true, y_pred))

mean_fpr = np.linspace(0,1,100)
fpr, tpr, thr = roc_curve(y_true, pred[:,1], pos_label=1)
roc_auc = auc(fpr, tpr)

              precision    recall  f1-score   support

          CN       0.90      0.79      0.84        24
          AD       0.55      0.75      0.63         8

    accuracy                           0.78        32
   macro avg       0.73      0.77      0.74        32
weighted avg       0.81      0.78      0.79        32

[[19  5]
 [ 2  6]]


### 75 - 85 y. o. group

In [ ]:
pred = pd.read_excel(r".../pred-age7585.xlsx")
pred = np.asarray(pred)

Y = pd.read_excel(r".../ground-age7585.xlsx")
Y = np.asarray(Y)

y_true = Y.argmax(axis=-1)
y_pred = pred.argmax(axis=-1)
report = classification_report(y_true, y_pred, target_names=['CN','AD'], output_dict=True)
print(classification_report(y_true, y_pred, target_names=['CN','AD']))
print(confusion_matrix(y_true, y_pred))

mean_fpr = np.linspace(0,1,100)
fpr, tpr, thr = roc_curve(y_true, pred[:,1], pos_label=1)
roc_auc = auc(fpr, tpr)

              precision    recall  f1-score   support

          CN       0.86      0.75      0.80         8
          AD       0.87      0.93      0.90        14

    accuracy                           0.86        22
   macro avg       0.86      0.84      0.85        22
weighted avg       0.86      0.86      0.86        22

[[ 6  2]
 [ 1 13]]


### >= 85 y. o. group

In [ ]:
pred = pd.read_excel(r".../pred-agemag85.xlsx")
pred = np.asarray(pred)

Y = pd.read_excel(r".../ground-agemag85.xlsx")
Y = np.asarray(Y)

y_true = Y.argmax(axis=-1)
y_pred = pred.argmax(axis=-1)
#report = classification_report(y_true, y_pred, target_names=['CN','AD'], output_dict=True)
#print(classification_report(y_true, y_pred, target_names=['CN','AD']))
print(confusion_matrix(y_true, y_pred))

#mean_fpr = np.linspace(0,1,100)
#fpr, tpr, thr = roc_curve(y_true, pred[:,1], pos_label=1)
#roc_auc = auc(fpr, tpr)

[[2]]


### <= 18 MMSE group

In [ ]:
pred = pd.read_excel(r".../pred-mmse18.xlsx")
pred = np.asarray(pred)

Y = pd.read_excel(r".../ground-mmse18.xlsx")
Y = np.asarray(Y)

y_true = Y.argmax(axis=-1)
y_pred = pred.argmax(axis=-1)
#report = classification_report(y_true, y_pred, target_names=['CN','AD'], output_dict=True)
#print(classification_report(y_true, y_pred, target_names=['CN','AD']))
print(confusion_matrix(y_true, y_pred))

#mean_fpr = np.linspace(0,1,100)
#fpr, tpr, thr = roc_curve(y_true, pred[:,1], pos_label=1)
#roc_auc = auc(fpr, tpr)

[[6]]


### 19 - 24 MMSE group

In [ ]:
pred = pd.read_excel(r".../pred-mmse1924.xlsx")
pred = np.asarray(pred)

Y = pd.read_excel(r".../ground-mmse1924.xlsx")
Y = np.asarray(Y)

y_true = Y.argmax(axis=-1)
y_pred = pred.argmax(axis=-1)
#report = classification_report(y_true, y_pred, target_names=['CN','AD'], output_dict=True)
#print(classification_report(y_true, y_pred, target_names=['CN','AD']))
print(confusion_matrix(y_true, y_pred))

#mean_fpr = np.linspace(0,1,100)
#fpr, tpr, thr = roc_curve(y_true, pred[:,1], pos_label=1)
#roc_auc = auc(fpr, tpr)

[[ 0  0]
 [ 1 14]]


### 25 MMSE group

In [ ]:
pred = pd.read_excel(r".../pred-mmse25.xlsx")
pred = np.asarray(pred)

Y = pd.read_excel(r".../ground-mmse25.xlsx")
Y = np.asarray(Y)

y_true = Y.argmax(axis=-1)
y_pred = pred.argmax(axis=-1)
#report = classification_report(y_true, y_pred, target_names=['CN','AD'], output_dict=True)
#print(classification_report(y_true, y_pred, target_names=['CN','AD']))
print(confusion_matrix(y_true, y_pred))

#mean_fpr = np.linspace(0,1,100)
#fpr, tpr, thr = roc_curve(y_true, pred[:,1], pos_label=1)
#roc_auc = auc(fpr, tpr)

[[0 0]
 [3 6]]


### 26 - 30 MMSE group

In [ ]:
pred = pd.read_excel(r".../pred-mmse2630.xlsx")
pred = np.asarray(pred)

Y = pd.read_excel(r".../ground-mmse2630.xlsx")
Y = np.asarray(Y)

y_true = Y.argmax(axis=-1)
y_pred = pred.argmax(axis=-1)
#report = classification_report(y_true, y_pred, target_names=['CN','AD'], output_dict=True)
#print(classification_report(y_true, y_pred, target_names=['CN','AD']))
print(confusion_matrix(y_true, y_pred))

#mean_fpr = np.linspace(0,1,100)
#fpr, tpr, thr = roc_curve(y_true, pred[:,1], pos_label=1)
#roc_auc = auc(fpr, tpr)

[[24  8]
 [ 0  0]]


### Overall

In [ ]:
pred = pd.read_excel(r".../pred-ADNI-2.xlsx")
pred = np.asarray(pred)

Y = pd.read_excel(r".../ground-ADNI-2.xlsx")
Y = np.asarray(Y)

y_true = Y.argmax(axis=-1)
y_pred = pred.argmax(axis=-1)
report = classification_report(y_true, y_pred, target_names=['CN','AD'], output_dict=True)
print(classification_report(y_true, y_pred, target_names=['CN','AD']))
print(confusion_matrix(y_true, y_pred))

mean_fpr = np.linspace(0,1,100)
fpr, tpr, thr = roc_curve(y_true, pred[:,1], pos_label=1)
roc_auc = auc(fpr, tpr)

              precision    recall  f1-score   support

          CN       0.79      0.70      0.74        33
          AD       0.73      0.82      0.77        33

    accuracy                           0.76        66
   macro avg       0.76      0.76      0.76        66
weighted avg       0.76      0.76      0.76        66

[[23 10]
 [ 6 27]]
